In [2]:
from src.data.consts import FILE_PATH
import pandas as pd
import zipfile

with zipfile.ZipFile(FILE_PATH, "r") as zip_ref:
    with zip_ref.open("filtered.tsv") as file:
        df = pd.read_csv(file, sep='\t')

In [3]:
tox_df = df[['reference', 'ref_tox']].copy()
tox_df = tox_df[:20000]

In [4]:
tox_df.head()

,reference,ref_tox
0,"If Alkar is flooding her with psychic waste, t...",0.014195
1,Now you're getting nasty.,0.065473
2,"Well, we could spare your life, for one.",0.213313
3,"Ah! Monkey, you've got to snap out of it.",0.053362
4,I've got orders to put her down.,0.009402


In [5]:
from sklearn.model_selection import train_test_split

X = tox_df['reference']  # тексты
y = tox_df['ref_tox']    # уровни токсичности

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
X_train_tfidf = X_train_tfidf.astype('float32')



In [7]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_tfidf.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tfidf.toarray(), y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_tfidf.toarray(), y_test))


In [10]:
batch_size = 32
epochs = 20

model.fit(train_dataset.batch(batch_size), epochs=epochs, validation_data=test_dataset.batch(batch_size))

Epoch 1/20
500/500 [==============================] - 6s 10ms/step - loss: 0.5792 - accuracy: 0.0000e+00 - val_loss: 0.5362 - val_accuracy: 0.0000e+00
Epoch 2/20
500/500 [==============================] - 5s 10ms/step - loss: 0.4480 - accuracy: 0.0000e+00 - val_loss: 0.5534 - val_accuracy: 0.0000e+00
Epoch 3/20
500/500 [==============================] - 5s 10ms/step - loss: 0.3712 - accuracy: 0.0000e+00 - val_loss: 0.6029 - val_accuracy: 0.0000e+00
Epoch 4/20
500/500 [==============================] - 5s 9ms/step - loss: 0.3087 - accuracy: 0.0000e+00 - val_loss: 0.6542 - val_accuracy: 0.0000e+00
Epoch 5/20
500/500 [==============================] - 5s 10ms/step - loss: 0.2634 - accuracy: 0.0000e+00 - val_loss: 0.7108 - val_accuracy: 0.0000e+00
Epoch 6/20
500/500 [==============================] - 6s 11ms/step - loss: 0.2375 - accuracy: 0.0000e+00 - val_loss: 0.7303 - val_accuracy: 0.0000e+00
Epoch 7/20
500/500 [==============================] - 5s 10ms/step - loss: 0.2225 - accuracy: 0

In [11]:
y_pred = model.predict(X_test_tfidf.toarray())

125/125 [==============================] - 0s 2ms/step


In [13]:
from sklearn.metrics import accuracy_score

# Преобразуйте истинные метки тестовых данных в бинарный формат, если они не таковы
y_test_binary = (y_test > 0.5).astype(int)
y_pred_binary = (y_pred > 0.5).astype(int)


# Оцените точность
accuracy = accuracy_score(y_test_binary, y_pred_binary)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.70
